In [84]:
import xgboost as xgb

In [13]:
#from sklearn.model_selection import train_test_split

In [85]:
import pandas as pd
import numpy as np

In [86]:
import matplotlib.pyplot as plt
import seaborn as SNS
%matplotlib inline

In [210]:
from sklearn.preprocessing import StandardScaler

In [87]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [88]:
model = xgb.XGBRegressor(n_estimators=1000)

In [119]:
#get covid restriction index
covid_data = pd.read_csv('data\owid-covid-data.csv')
covid_data = covid_data[covid_data['iso_code'] == 'CZE']
restriction_index = pd.DataFrame()
restriction_index = covid_data[['date','stringency_index']]
restriction_index = restriction_index.rename(columns = {'date':'ds'})
restriction_index['ds']=  pd.to_datetime(restriction_index['ds'])
restriction_index.head()

,ds,stringency_index
49132,2020-03-01,19.44
49133,2020-03-02,19.44
49134,2020-03-03,19.44
49135,2020-03-04,19.44
49136,2020-03-05,19.44


In [120]:
df = pd.read_csv('prophetdf.csv')
df['ds']=  pd.to_datetime(df['ds'])

In [121]:
#feats month, dayofyear, week, year, holiday_season(nov 1 - dec 23), holiday, covid_lockdown, weekday

df['Day'] = df['ds'].dt.day
df['Week'] = df['ds'].dt.week
df['Month'] = df['ds'].dt.month
df['Year'] = df['ds'].dt.year
df['DayofYear'] = df['ds'].dt.dayofyear
df['DayofWeek'] = df['ds'].dt.dayofweek

C:\Users\Alex\AppData\Local\Temp\ipykernel_447732\619729888.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['Week'] = df['ds'].dt.week


In [249]:
#join stringency index to full dataset, fill nan with zero
data = pd.merge(df,restriction_index,on='ds',how='left')
data = data.fillna(0)

In [175]:
forecast.columns.values

array(['ds', 'trend', 'yhat', 'weekly', 'yearly', 'holidays'],
      dtype=object)

In [250]:
#prophet prediction
forecast = pd.read_csv('prophet_forecast.csv')
forecast = forecast[['ds', 'trend','yhat','weekly', 'yearly','holidays']]

In [251]:
forecast.head()

,ds,trend,yhat,weekly,yearly,holidays
0,2019-07-02,42910.668771,30871.938406,0.038724,-0.319278,0.000000
1,2019-07-03,42941.568091,29545.852006,0.004868,-0.316820,0.000000
2,2019-07-04,42972.467412,27928.754660,-0.036591,-0.313487,0.000000
3,2019-07-05,43003.366732,25372.628485,-0.056505,-0.309357,-0.044123
4,2019-07-06,43034.266053,23177.983567,-0.143097,-0.304512,-0.013797


In [252]:
forecast['weekly'].value_counts()

-0.056505    2
-0.036591    2
 0.038724    1
 0.117664    1
-0.036591    1
            ..
 0.074937    1
 0.117664    1
 0.038724    1
 0.004868    1
-0.036591    1
Name: weekly, Length: 921, dtype: int64

In [253]:
forecast['ds']=  pd.to_datetime(forecast['ds'])
data = pd.merge(data,forecast,on='ds',how='left')
data.head()

,ds,y,Day,Week,Month,Year,DayofYear,DayofWeek,stringency_index,trend,yhat,weekly,yearly,holidays
0,2019-06-01,4087.236534,1,22,6,2019,152,5,0.0,NaN,NaN,NaN,NaN,NaN
1,2019-06-02,65.848327,2,22,6,2019,153,6,0.0,NaN,NaN,NaN,NaN,NaN
2,2019-06-03,1364.979529,3,23,6,2019,154,0,0.0,NaN,NaN,NaN,NaN,NaN
3,2019-06-04,68385.521621,4,23,6,2019,155,1,0.0,NaN,NaN,NaN,NaN,NaN
4,2019-06-05,65135.657001,5,23,6,2019,156,2,0.0,NaN,NaN,NaN,NaN,NaN


In [254]:
#remove outliers and first month to match forecast df
data = data.drop(index=[366,468,774,314,957]) #180, 213


In [159]:
data.head()

,ds,y,Day,Week,Month,Year,DayofYear,DayofWeek,stringency_index,trend,yhat,weekly,yearly,holidays
0,2019-06-01,4087.236534,1,22,6,2019,152,5,0.0,NaN,NaN,NaN,NaN,NaN
1,2019-06-02,65.848327,2,22,6,2019,153,6,0.0,NaN,NaN,NaN,NaN,NaN
2,2019-06-03,1364.979529,3,23,6,2019,154,0,0.0,NaN,NaN,NaN,NaN,NaN
3,2019-06-04,68385.521621,4,23,6,2019,155,1,0.0,NaN,NaN,NaN,NaN,NaN
4,2019-06-05,65135.657001,5,23,6,2019,156,2,0.0,NaN,NaN,NaN,NaN,NaN


In [255]:
#data['Day'] = data['ds'].dt.day
data['Week'] = data['ds'].dt.week
data['Month'] = data['ds'].dt.month
data['Year'] = data['ds'].dt.year
data['DayofYear'] = data['ds'].dt.dayofyear
#data['DayofWeek'] = data['ds'].dt.dayofweek

C:\Users\Alex\AppData\Local\Temp\ipykernel_447732\775285933.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data['Week'] = data['ds'].dt.week


In [256]:
#holiday_season(nov 1 - dec 23)
def holiday_season(x):
    if 305 <= x <= 357:
        return 1
    return 0
data['holiday_season'] = data['DayofYear'] .apply(holiday_season)

In [257]:
data.head()
#onehot encode DayofWeek

,ds,y,Day,Week,Month,Year,DayofYear,DayofWeek,stringency_index,trend,yhat,weekly,yearly,holidays,holiday_season
0,2019-06-01,4087.236534,1,22,6,2019,152,5,0.0,NaN,NaN,NaN,NaN,NaN,0
1,2019-06-02,65.848327,2,22,6,2019,153,6,0.0,NaN,NaN,NaN,NaN,NaN,0
2,2019-06-03,1364.979529,3,23,6,2019,154,0,0.0,NaN,NaN,NaN,NaN,NaN,0
3,2019-06-04,68385.521621,4,23,6,2019,155,1,0.0,NaN,NaN,NaN,NaN,NaN,0
4,2019-06-05,65135.657001,5,23,6,2019,156,2,0.0,NaN,NaN,NaN,NaN,NaN,0


In [258]:
#offset
data['y1'] = data['y'].shift(1)

In [259]:
data = data[30:]
data

,ds,y,Day,Week,Month,Year,DayofYear,DayofWeek,stringency_index,trend,yhat,weekly,yearly,holidays,holiday_season,y1
30,2019-07-02,29825.892188,2,27,7,2019,183,1,0.00,42910.668771,30871.938406,0.038724,-0.319278,0.000000,0,30217.879866
31,2019-07-03,30148.139918,3,27,7,2019,184,2,0.00,42941.568091,29545.852006,0.004868,-0.316820,0.000000,0,29825.892188
32,2019-07-04,28886.698448,4,27,7,2019,185,3,0.00,42972.467412,27928.754660,-0.036591,-0.313487,0.000000,0,30148.139918
33,2019-07-05,29277.998737,5,27,7,2019,186,4,0.00,43003.366732,25372.628485,-0.056505,-0.309357,-0.044123,0,28886.698448
34,2019-07-06,21320.833411,6,27,7,2019,187,5,0.00,43034.266053,23177.983567,-0.143097,-0.304512,-0.013797,0,29277.998737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,2022-01-09,101228.700023,9,1,1,2022,9,6,37.76,113616.345360,117891.200706,0.074937,-0.037312,0.000000,0,92744.263348
953,2022-01-10,107064.467839,10,2,1,2022,10,0,37.75,113715.142266,124649.179829,0.117664,-0.021511,0.000000,0,101228.700023
954,2022-01-11,110698.874419,11,2,1,2022,11,1,37.75,113813.939173,117475.785954,0.038724,-0.006550,0.000000,0,107064.467839
955,2022-01-12,100503.117963,12,2,1,2022,12,2,37.74,113912.736079,115301.625990,0.004868,0.007325,0.000000,0,110698.874419


In [260]:
#set date to index
data = data.reset_index(drop=True)

In [282]:
X = data.drop(columns = ['ds','y'])
y =  data['y']

In [283]:
X_train = X[:-30]
X_test = X[-30:]
y_train = y[:-30]
y_test = y[-30:]

In [212]:
#scaler= StandardScaler()

In [245]:
'''scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)'''

In [295]:
X_test.iloc[2]

Day                     17.000000
Week                    50.000000
Month                   12.000000
Year                  2021.000000
DayofYear              351.000000
DayofWeek                4.000000
stringency_index        39.690000
trend               111344.016511
yhat                110173.161376
weekly                  -0.056505
yearly                   0.045990
holidays                 0.000000
holiday_season           1.000000
y1                  138778.306261
Name: 895, dtype: float64

In [289]:
X_train

,Day,Week,Month,Year,DayofYear,DayofWeek,stringency_index,trend,yhat,weekly,yearly,holidays,holiday_season,y1
0,2,27,7,2019,183,1,0.00,42910.668771,30871.938406,0.038724,-0.319278,0.000000,0,30217.879866
1,3,27,7,2019,184,2,0.00,42941.568091,29545.852006,0.004868,-0.316820,0.000000,0,29825.892188
2,4,27,7,2019,185,3,0.00,42972.467412,27928.754660,-0.036591,-0.313487,0.000000,0,30148.139918
3,5,27,7,2019,186,4,0.00,43003.366732,25372.628485,-0.056505,-0.309357,-0.044123,0,28886.698448
4,6,27,7,2019,187,5,0.00,43034.266053,23177.983567,-0.143097,-0.304512,-0.013797,0,29277.998737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,10,49,12,2021,344,4,45.34,110652.438165,141747.271991,-0.056505,0.337519,0.000000,1,170134.500329
889,11,49,12,2021,345,5,45.34,110751.235072,127595.763459,-0.143097,0.295190,0.000000,1,152729.321366
890,12,49,12,2021,346,6,45.33,110850.031978,147116.847023,0.074937,0.252233,0.000000,1,148970.635673
891,13,50,12,2021,347,0,45.32,110948.828885,147211.230378,0.117664,0.209175,0.000000,1,180337.688971


In [296]:
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [317]:
X_test['y1'][893]

183912.3136204

In [326]:
y_pred = []
for x in range(len(X_test)):
    test = X_test[x:x+1]
    pred = model.predict(test)
    X_test['y1'][x+894] = float(pred)
    y_pred.append(float(pred))


C:\Users\Alex\AppData\Local\Temp\ipykernel_447732\2757190253.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['y1'][x+894] = float(pred)


In [327]:
y_pred

[188734.265625,
 164829.9375,
 154595.671875,
 121124.9765625,
 145428.59375,
 145160.734375,
 131924.796875,
 132199.109375,
 125395.4921875,
 106022.1484375,
 109530.3671875,
 139937.96875,
 143052.984375,
 134827.765625,
 124818.84375,
 125442.4453125,
 92848.5546875,
 99016.484375,
 121146.4140625,
 119354.8046875,
 128123.7421875,
 131626.15625,
 140042.609375,
 130624.3359375,
 115007.125,
 150630.171875,
 164587.34375,
 155482.078125,
 154238.828125,
 159058.171875]

In [328]:
np.sqrt(mean_squared_error(y_test,y_pred))

40391.10283738487

In [329]:
data['yhat'][-30:]

893    125559.711120
894    116572.737933
895    110173.161376
896     96581.426725
897    117234.523304
898    118678.725333
899    106885.837265
900    100492.746190
901     93632.999568
902     79657.624656
903     76670.991090
904    104045.635878
905    106107.393556
906     97021.031772
907     93357.507048
908     89172.371986
909     78653.186429
910     69924.633081
911    104954.584380
912    111323.447990
913    104106.797290
914    102108.093329
915     99330.053625
916     99036.496545
917     91180.400508
918    117891.200706
919    124649.179829
920    117475.785954
921    115301.625990
922    112108.702382
Name: yhat, dtype: float64

In [330]:
X_test

,Day,Week,Month,Year,DayofYear,DayofWeek,stringency_index,trend,yhat,weekly,yearly,holidays,holiday_season,y1
893,15,50,12,2021,349,2,39.73,111146.422698,125559.711120,0.004868,0.124811,0.000000,1,183912.313620
894,16,50,12,2021,350,3,39.71,111245.219604,116572.737933,-0.036591,0.084481,0.000000,1,188734.265625
895,17,50,12,2021,351,4,39.69,111344.016511,110173.161376,-0.056505,0.045990,0.000000,1,164829.937500
896,18,50,12,2021,352,5,39.69,111442.813417,96581.426725,-0.143097,0.009742,0.000000,1,154595.671875
897,19,50,12,2021,353,6,39.68,111541.610324,117234.523304,0.074937,-0.023899,0.000000,1,121124.976562
898,20,51,12,2021,354,0,39.67,111640.407230,118678.725333,0.117664,-0.054619,0.000000,1,145428.593750
899,21,51,12,2021,355,1,39.65,111739.204137,106885.837265,0.038724,-0.082159,0.000000,1,145160.734375
900,22,51,12,2021,356,2,39.64,111838.001043,100492.746190,0.004868,-0.106311,0.000000,1,131924.796875
901,23,51,12,2021,357,3,39.63,111936.797950,93632.999568,-0.036591,-0.126928,0.000000,1,132199.109375
902,24,51,12,2021,358,4,39.63,112035.594856,79657.624656,-0.056505,-0.143917,-0.088575,0,125395.492188


In [332]:
y_pred

[188734.265625,
 164829.9375,
 154595.671875,
 121124.9765625,
 145428.59375,
 145160.734375,
 131924.796875,
 132199.109375,
 125395.4921875,
 106022.1484375,
 109530.3671875,
 139937.96875,
 143052.984375,
 134827.765625,
 124818.84375,
 125442.4453125,
 92848.5546875,
 99016.484375,
 121146.4140625,
 119354.8046875,
 128123.7421875,
 131626.15625,
 140042.609375,
 130624.3359375,
 115007.125,
 150630.171875,
 164587.34375,
 155482.078125,
 154238.828125,
 159058.171875]

In [333]:
y_test

893    184233.647435
894    138778.306261
895    109987.895659
896     89303.723054
897     95343.122476
898    102887.905090
899     86748.382460
900     74357.172142
901     74029.801498
902     59059.391196
903     84937.096045
904     95617.620952
905    118019.757473
906    106367.443052
907    110688.021437
908     82701.836914
909     44111.592040
910     54104.788051
911     93466.279742
912    102294.937063
913     98104.353685
914    104320.580030
915    101522.795968
916     96196.124898
917     92744.263348
918    101228.700023
919    107064.467839
920    110698.874419
921    100503.117963
922     97576.039846
Name: y, dtype: float64